<a href="https://colab.research.google.com/github/ahmadhajmosa/Machine-learning-labs/blob/AlessandroFornasier/Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab on Machine Learning and Applications in Intelligent Vehicles
## Session 1: Introduction


#Session 2: 05.06 - 13:00 - 14:30 :




## Intro:

Tensorflow is a powerful framework for implementing and deploying large-scale deep learning models. Recently, it has been widely used in both reasearch and production. TF objective is to combine scale and flexibility.

In the past session, we will learning the following:

1. TF programming stack
2. TF programming concepts including computatoin graphs, operations and sessions. 
3. Implementation of linear regression
4. Implementation of feed-forward neural networks

## TF stack:

TensorFlow is a framework composed of two core building blocks — a library for defining computational graphs and a runtime for executing such graphs on a variety of different hardware


![alt text](https://www.tensorflow.org/images/layers.png)


Before goining into details about the stack, let us talk about computational graphs.

### Computational Graphs

A directed graph is a data structure consisting of nodes (vertices) and edges. It’s a set of vertices connected pairwise by directed edges.

Graphs come in many shapes and sizes and are used to solve many real-life problems, such as representing networks including telephone networks, circuit networks, road networks, and even social networks. 
![alt text](https://cdn-images-1.medium.com/max/800/1*V6aYjD3AxDbEKYahkGqVQw.png)

TensorFlow uses directed graphs internally to represent computations, and they call this data flow graphs (or computational graphs).

The nodes in TF data flow graph mostly represents operations, variables and placeholders.

Take for example the following operation:
![alt text](https://cdn-images-1.medium.com/max/800/1*6E3sfit6DCeJ9mOz17g4bA.png)

To create a computational graph out of this program, we create nodes for each of the operations in our program, along with the input variables a and b. In fact, a and b could be constants if they don’t change. If one node is used as the input to another operation we draw a directed arrow that goes from one node to another.

The computational graph for this program might look like this:
![alt text](https://cdn-images-1.medium.com/max/800/1*vPb9E0Yd1QUAD0oFmAgaOw.png)

Operations create or manipulate data according to specific rules. In TensorFlow those rules are called Ops, short for operations. Variables on the other hand represent shared, persistent state that can be manipulated by running Ops on those variables.

The questions now what are the advantages of representing operations as directed graphs: The main advantage of using directed graphs is the ability to do **parallelism** and what is called **dependency driving scheduling**. 
For example, consider again the follwoing code:
![alt text](https://cdn-images-1.medium.com/max/800/1*6E3sfit6DCeJ9mOz17g4bA.png)
At the most fundamental level, most computer programs are mainly composed of two things — primitive operations and an order in which these operations are executed, often sequentially, line by line. This means we would first multiply a and b and only when this expression was evaluated we would take their sum. Computational graphs on the otherhand, exclusively specify the dependencies across the operations.
If we look at our computational graph we see that we could execute the multiplication and addition in parallel. That’s because these two operations do not depend on each other.
 So we can use the topology of the graph to drive the scheduling of operations and execute them in the most efficient manner, e.g. using multiple GPUs on a single machine or even distribute the execution across multiple machines.
 Another key advantage is portability. The graph is a language-independent representation of our code. So we can build the graph in Python, save the model (TensorFlow uses protocol buffers), and restore the model in a different language, say C++, if you want to go really fast.
 
 

--------------------------------
# References:

https://medium.com/@d3lm/understand-tensorflow-by-mimicking-its-api-from-scratch-faa55787170d

https://www.tensorflow.org/guide/extend/architecture

https://www.tensorflow.org/guide/low_level_intro

  
 






# placeholder: tensors are feeded externaly for example inputs tensors + output tensors

# variables : tensors represent the parameters of the network/graph ie. nn weights

In [0]:
import tensorflow as tf
import numpy as np

# Parameters
learning_rate = 0.001
training_iters = 2000
batch_size = 128

# Network Parameters
num_inputs = 3
num_outputs = 4
num_samples= 10

# Training data
x_gr = np.random.rand(num_samples,num_inputs)
y_gr = np.random.rand(num_samples,num_outputs)

# tf Graph input
x = tf.placeholder(tf.float32, [None, num_inputs])
y = tf.placeholder(tf.float32, [None, num_outputs])

# weights 
w_1 = tf.Variable(tf.random_normal([num_inputs,num_outputs ]))

# model (matmul is the dotproduct)
y_p = tf.matmul(x, w_1)

# cost
cost = tf.reduce_mean(tf.pow(y-y_p,2))

# optimisation 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initalizing the graph and the weights
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(10):
        #first run the optimizer (one iteration of the gradient descent)
        #the feed_dict is a dictionary of the data that has to be unsed instead
        #of the placeholders
        sess.run(optimizer, feed_dict={x: x_gr, y: y_gr}) 

        pr_cost = sess.run(cost, feed_dict={x: x_gr,y: y_gr})
    
        print('iter: ',i, 'cost: ', pr_cost)
    
    y_p_p = sess.run(y_p, feed_dict={x: x_gr, y: y_gr})
    
    print('predicted ', y_p_p)
    print('real ', y_gr)



In [0]:
import tensorflow as tf
import numpy as np

# Parameters
learningRate = 0.001
trainingIterations = 2000
batchSize = 128

# Network Parameters
nInputs = 3
nOutputs = 4
nHiddenLayerNeurons = np.array([4,10])
nSamples= 10

# Training data (GroundTruth)
xGroundTruth = np.random.rand(nSamples,nInputs)
yGroundTruth = np.random.rand(nSamples,nOutputs)

# TensorFlow Graph input/output placeholders
x = tf.placeholder(tf.float32, [None, nInputs])
y = tf.placeholder(tf.float32, [None, nOutputs])

# Weights 
W1 = tf.Variable(tf.random_normal([nInputs,nHiddenLayerNeurons[0]]))
W2 = tf.Variable(tf.random_normal([nHiddenLayerNeurons[0],nHiddenLayerNeurons[1]]))
W3 = tf.Variable(tf.random_normal([nHiddenLayerNeurons[1],nOutputs]))

# Biases
B1 = tf.Variable(tf.random_normal([nHiddenLayerNeurons[0]]))
B2 = tf.Variable(tf.random_normal([nHiddenLayerNeurons[1]]))
B3 = tf.Variable(tf.random_normal([nOutputs]))

# Model
H1 = tf.nn.sigmoid(tf.add(tf.matmul(x,W1),B1))
H2 = tf.nn.sigmoid(tf.add(tf.matmul(H2,W2),B2))
Y = tf.add(tf.matmul(H2,W3),B3)

# Cost ... complete from here ...
cost = tf.reduce_mean(tf.pow(y-y_p,2))

# optimisation 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initalizing the graph and the weights
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(10):
        #first run the optimizer (one iteration of the gradient descent)
        #the feed_dict is a dictionary of the data that has to be unsed instead
        #of the placeholders
        sess.run(optimizer, feed_dict={x: x_gr, y: y_gr}) 

        pr_cost = sess.run(cost, feed_dict={x: x_gr,y: y_gr})
    
        print('iter: ',i, 'cost: ', pr_cost)
    
    y_p_p = sess.run(y_p, feed_dict={x: x_gr, y: y_gr})
    
    print('predicted ', y_p_p)
    print('real ', y_gr)



4


In [18]:
# Libraries import
import tensorflow as tf
import numpy as np

# Import MNIST data input (28x28 image --> mnist.train.images.shape)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

#import imshow
from matplotlib.pyplot import imshow
%matplotlib inline

# Extract the traning, test and validation data and labels
xTrain = mnist.train.images
yTrain = mnist.train.labels
xTest = mnist.test.images
yTest = mnist.test.labels
xValidation = mnist.validation.images
yValidation = mnist.validation.labels

# Parameters
learningRate = 0.001
trainingIterations = 2000
batchSize = 128

# Network Parameters
nInputs =  mnist.train.images[0].size
nOutputs = mnist.train.labels[0].size
nHiddenLayerNeurons = np.array([100,100])
nSamples= 10

# Training data (GroundTruth)
xGroundTruth = np.random.rand(nSamples,nInputs)
yGroundTruth = np.random.rand(nSamples,nOutputs)

# TensorFlow Graph input/output placeholders
x = tf.placeholder(tf.float32, [None, nInputs])
y = tf.placeholder(tf.float32, [None, nOutputs])

# Weights 
W1 = tf.Variable(tf.random_normal([nInputs,nHiddenLayerNeurons[0]]))
W2 = tf.Variable(tf.random_normal([nHiddenLayerNeurons[0],nHiddenLayerNeurons[1]]))
W3 = tf.Variable(tf.random_normal([nHiddenLayerNeurons[1],nOutputs]))

# Biases
B1 = tf.Variable(tf.random_normal([nHiddenLayerNeurons[0]]))
B2 = tf.Variable(tf.random_normal([nHiddenLayerNeurons[1]]))
B3 = tf.Variable(tf.random_normal([nOutputs]))

# Model
H1 = tf.nn.sigmoid(tf.add(tf.matmul(x,W1),B1))
H2 = tf.nn.sigmoid(tf.add(tf.matmul(H1,W2),B2))
Y = tf.add(tf.matmul(H2,W3),B3)

# Model Evaluation (tf.equal compare the 2 matricies and returns another matrix
# with every values equal to true if the 2 value in the matricies are the same
# false otherwise)
nCorrectPrediction = tf.equal(tf.argmax(Y,1), tf.argmax(y,1))

# Cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y, labels=y))

# Accurancy
accurancy = tf.reduce_mean(tf.cast(nCorrectPrediction, tf.float32))

# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(cost)

# Initalizing the graph and the weights
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
  
    # Session initialization
    sess.run(init)
    
    for iteration in range(trainingIterations):
      
      # Take the samples from the dataset of the specified batchSize
      xTrainBatch, yTrainBatch = mnist.train.next_batch(batchSize)
      
      # Take the samples from the dataset of the specified batchSize
      xTestBatch, yTestBatch = mnist.test.next_batch(batchSize)
      
      # First run the optimizer (one iteration of the gradient descent)
      # the feed_dict is a dictionary of the data that has to be unsed
      # insteadof the placeholders
      sess.run(optimizer, feed_dict={x: xTrainBatch, y: yTrainBatch}) 

      # Evaluate the cost and accurancy during the traning process
      trainCost, trainAccurancy = sess.run([cost, accurancy], feed_dict={x: xTrainBatch, y: yTrainBatch})
      
      # Evaluate the cost and accurancy during the traning process
      testCost, testAccurancy = sess.run([cost, accurancy], feed_dict={x: xTestBatch, y: yTestBatch})
      
      print('\nIteration: ', iteration, 'trainCost: ', trainCost, 'trainAccurancy: ', trainAccurancy, 'testCost: ', testCost, 'testAccurancy: ', testAccurancy )


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz

Iteration:  0 trainCost:  14.767424 trainAccurancy:  0.0859375 testCost:  15.287432 testAccurancy:  0.0625

Iteration:  1 trainCost:  13.770051 trainAccurancy:  0.125 testCost:  13.611641 testAccurancy:  0.1015625

Iteration:  2 trainCost:  13.092819 trainAccurancy:  0.1015625 testCost:  13.796176 testAccurancy:  0.078125

Iteration:  3 trainCost:  13.455248 trainAccurancy:  0.0703125 testCost:  13.493982 testAccurancy:  0.1015625

Iteration:  4 trainCost:  12.022712 trainAccurancy:  0.1015625 testCost:  14.158468 testAccurancy:  0.078125

Iteration:  5 trainCost:  12.252884 trainAccurancy:  0.109375 testCost:  12.264145 testAccurancy:  0.09375

Iteration:  6 trainCost:  12.706514 trainAccurancy:  0.09375 testCost:  11.409385 testAccurancy:  0.1171875

Iteration:  7 trainCost:  11.6320505 trainAccu